<a href="https://colab.research.google.com/github/YeweiYuan/Quantum-Pricing/blob/main/qfin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

量子期权定价
---



---
蒙特卡罗方法和blm



In [ ]:
import numpy as np
from scipy.stats import norm
from numpy.random import standard_normal

In [74]:
class Sampling:
  def __init__(self,N=10000,T=0.5,S_0=5.29,K=6,Rate=0.04,Sigma=0.24): #初始化六个月的股票期权，标的资产股票价格为5.29元，期权执行价格为6元，年化无风险利率和年化波动率分别为4%和24%，模拟10000次
    self.N = N # sample size
    self.T = T # time
    self.S_0 = S_0 # init price
    self.Rate = Rate # rate
    self.Sigma = Sigma # volatility
    self.K = K # strike price

  def callMonteCarlo(self): #蒙特卡罗法，看涨期权
    z = standard_normal(self.N)
    S_t = self.S_0*np.exp((self.Rate-0.5*self.Sigma**2)*self.T+self.Sigma*z*np.sqrt(self.T))
    call = np.maximum(0,S_t-self.K)
    res = sum(call)*(np.exp(-self.Rate*self.T))/self.N
    return res
  
  def callBLM(self): #BSM模型，看涨期权
    d1 = (np.log(self.S_0/self.K)+(self.Rate*self.T))/(self.Sigma*np.sqrt(self.T))+0.5*self.Sigma*np.sqrt(self.T)
    d2 = d1-self.Sigma*np.sqrt(self.T)
    call = self.S_0*norm.cdf(d1,0.0,1.0)-self.K*np.exp(-self.Rate*self.T)*norm.cdf(d2,0.0,1.0)
    return call

In [75]:
test = Sampling()

In [76]:
print('MonteCarlo: %f \nBLM: %f'%(test.callMonteCarlo(),test.callBLM()))

MonteCarlo: 0.156348 
BLM: 0.153154


---
量子方法

In [60]:
!pip install pyqpanda

     |████████████████████████████████| 20.8MB 1.3MB/s 


In [61]:
import pyqpanda.pyQPanda as pq
import math

In [62]:
import numpy as np
from scipy.stats import norm
from numpy.random import standard_normal

In [120]:
class Qsampling:
  def __init__(self,N=3,M=3,T=0.5,S_0=5.29,K=6,Rate=0.04,Sigma=0.24,machineType=pq.QMachineType.CPU): #3个用于拟合分布的qubits，3个用于相位估计的qubits
    self.N = N
    self.M = M
    self.T = T # time
    self.S_0 = S_0 # init price
    self.Rate = Rate # rate
    self.Sigma = Sigma # volatility
    self.K = K # strike price
    self.m_machine = pq.init_quantum_machine(machineType)
    self.m_qlist = self.m_machine.qAlloc_many(N+M)
    self.m_clist = self.m_machine.cAlloc_many(N+M)
    self.m_prog = pq.QProg()

  def __del__(self):
    pq.destroy_quantum_machine(self.m_machine)

  def sample(self):
    x_max = np.sqrt(self.T)*3 #这里取3倍标准差涵盖99.7的分布
    delta = 2*x_max/(2**self.N-1)
    p_j = []
    for j in range(2**self.N):
      p_j.append(norm.pdf(-x_max+j*delta))
    p_j = p_j/sum(p_j)
    '''
    未完成的手动量子态振幅编码程序
    p_mk = [[] for i in range(self.N)]
    for m in range(self.N):
      for k in range(2**(m+1)):
        p_mk[m].append(sum(p_j[k*2**(self.N-m-1):(k+1)*2**(self.N-m-1)]))
 
    theta_mk = [[] for i in range(self.N-1)]
    for m in range(self.N-1):
      for k in range(2**(m+1)):
        theta_mk[m].append(np.arccos(np.sqrt(p_mk[m+1][2*k]/p_mk[m][k])))
    '''
    qlist = self.m_qlist
    cir_encode = pq.amplitude_encode(qlist[0:self.N],p_j)
    print(p_j)

    #self.m_prog.insert(pq.Measure(qlist[0:self.N], self.m_clist[0:self.N]))
    #res = pq.run_with_configuration(self.m_prog,m_clist,1000)

    res = pq.prob_run_tuple_list(self.m_prog,qlist[0:N],N)
    print(res)

In [126]:
Qtest = Qsampling()
Qtest.sample()

[0.02584214 0.07779269 0.16218559 0.23417958 0.23417958 0.16218559
 0.07779269 0.02584214]


RuntimeError: ignored

In [113]:
Qtest.sample()
res = pq.prob_run_tuple_list(Qtest.m_prog,Qtest.m_qlist[0:N],N)
print(res)

[0.02584214 0.07779269 0.16218559 0.23417958 0.23417958 0.16218559
 0.07779269 0.02584214]


RuntimeError: ignored